In [ ]:
"""
Источник: https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/Online_Noise_Augmentation.ipynb
"""

#Установка необходимых пакетов
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]
!pip install librosa==0.9.2
!pip install json
!pip install soundfile==0.12.1

In [ ]:
import json
from nemo.collections.asr.parts.preprocessing import perturb, segment
import glob
import soundfile as sf
import os
import librosa

* Перейдите в раздел "Общий доступ" в Google Диске.
* Выберите SPbU_smart-assistant
* Щелкните ее правой кнопкой мыши и выберите Добавить ярлык на диск.
* Появится всплывающее окно, выберите MyDrive и нажмите Добавить ярлык.
* Теперь перейдите в блокнот Google Colab и подключитесь к Google Диску с помощью кнопки


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Загрузка набор данных. Это займет несколько мгновений...
print("******")
if not os.path.exists("./raw"):
  os.makedirs("./raw")

if not os.path.exists("./noises"):
  os.makedirs("./noises")

for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  !cp -v $audio_path "./noises"

for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/raw/*.*"):
  !cp -v $audio_path "./raw"
print("Конец.\n******")

******
'/content/drive/MyDrive/SPbU_smart-assistant/noises/1.wav' -> './noises/1.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/2.wav' -> './noises/2.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/3.wav' -> './noises/3.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/6.wav' -> './noises/6.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/7.wav' -> './noises/7.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/4.wav' -> './noises/4.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/5.wav' -> './noises/5.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/raw/Audio3.wav' -> './raw/Audio3.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/raw/Audio2.wav' -> './raw/Audio2.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/raw/Audio5.wav' -> './raw/Audio5.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/raw/Audio6.wav' -> './raw/Audio6.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/raw/Audio4.wav' -> './raw/Audio4.wav'
'/content/drive/MyDrive/S

In [6]:
#загрузка первого аудио для определения sr
example = glob.glob("/content/raw/*.*")[0]
sample, sr = librosa.core.load(example)

In [7]:
#Вспомогательная функция из Nemo
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

In [8]:
#Функция создания манифеста шумов
def write_manifest(filepath, data_dir='.', manifest_name='noise_manifest', duration_max=1e9, duration_stride=1.0):
              
    with open(os.path.join(data_dir, manifest_name + '.json'), 'w') as fout:
        y, sr = librosa.load(filepath)
        duration = librosa.get_duration(y=y, sr=sr)

        offsets = []
        durations = []

        if duration > duration_max:
            current_offset = 0.0

            while current_offset < duration:
                difference = duration - current_offset
                segment_duration = min(duration_max, difference)

                offsets.append(current_offset)
                durations.append(segment_duration)

                current_offset += duration_stride

        else:
            offsets.append(0.0)
            durations.append(duration)


        for duration, offset in zip(durations, offsets):
            metadata = {
                'audio_filepath': filepath,
                'duration': duration,
                'label': 'noise',
                'text': '_',  # for compatibility with ASRAudioText collection
                'offset': offset,
            }

            json.dump(metadata, fout)
            fout.write('\n')
            fout.flush()

        name = noise_path.split('/')[-1]
        print(f"Wrote {len(durations)} segments for filename {name}")
            
    print("Finished preparing manifest !")

In [9]:
#Манифесты для всех аудио шумов
if not os.path.exists("./manifests"):
  os.makedirs("./manifests")

for noise_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  noise_sample, noise_sr = librosa.core.load(noise_path)
  sf.write(noise_path, noise_sample, samplerate=noise_sr)
  noise_sample, noise_sr = librosa.core.load(noise_path)
  name = noise_path.split('/')[-1][-5]
  write_manifest(noise_path, data_dir='./manifests', manifest_name=f'noise_{name}s', duration_stride=1.0, duration_max=30.0)

Wrote 48 segments for filename 1.wav
Finished preparing manifest !
Wrote 35 segments for filename 2.wav
Finished preparing manifest !
Wrote 43 segments for filename 3.wav
Finished preparing manifest !
Wrote 94 segments for filename 6.wav
Finished preparing manifest !
Wrote 103 segments for filename 7.wav
Finished preparing manifest !
Wrote 123 segments for filename 4.wav
Finished preparing manifest !
Wrote 168 segments for filename 5.wav
Finished preparing manifest !


In [10]:
#Вероятности аугументаций
probas = [1.0, 1.0, 1.0]

for i in range(14):
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}s.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for path in glob.glob("./raw/*.*"):
    sample_segment = load_audio(path)
    audio_augmentations.perturb(sample_segment)
    name = path.split('/')[-1][:-4]

    sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{name}_{i}.wav", 
             sample_segment.samples, samplerate=sr, subtype='PCM_24')

[NeMo I 2023-04-01 13:08:33 collections:193] Dataset loaded with 48 files totalling 0.28 hours
[NeMo I 2023-04-01 13:08:33 collections:194] 0 files were filtered totalling 0.00 hours


[NeMo W 2023-04-01 13:08:34 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/nemo/collections/asr/parts/preprocessing/segment.py:449: RuntimeWarning: divide by zero encountered in log10
      return 10 * np.log10(mean_square)
    


[NeMo I 2023-04-01 13:08:34 collections:193] Dataset loaded with 35 files totalling 0.17 hours
[NeMo I 2023-04-01 13:08:34 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 13:08:36 collections:193] Dataset loaded with 43 files totalling 0.23 hours
[NeMo I 2023-04-01 13:08:36 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 13:08:37 collections:193] Dataset loaded with 123 files totalling 0.90 hours
[NeMo I 2023-04-01 13:08:37 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 13:08:38 collections:193] Dataset loaded with 168 files totalling 1.28 hours
[NeMo I 2023-04-01 13:08:38 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 13:08:39 collections:193] Dataset loaded with 94 files totalling 0.66 hours
[NeMo I 2023-04-01 13:08:39 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 13:08:40 collections:193] Dataset loaded with 103 files totallin